# Kalman Filter and Regression Methods Explained

This notebook combines code implementations with mathematical explanations using LaTeX.

In [ ]:
using Pkg
Pkg.add(["LinearAlgebra", "Plots", "DataFrames", "GLM", "StatsModels", "Statistics", "StatsPlots", "LaTeXStrings"])
using LinearAlgebra
using Plots
using GLM, DataFrames, StatsModels, Statistics, StatsPlots
using LaTeXStrings

## 1. Linear Regression

### Mathematical Formulation

The basic linear regression problem can be formulated as:

$$A = \begin{pmatrix}
 1 & x_0  \\
 \vdots & \vdots \\
1 & x_n \\
\end{pmatrix}$$

We solve the system:

$$Ax = b \rightarrow A^TAx=A^Tb \rightarrow \hat{x} = (A^TA)^{-1}A^Tb$$

Where:
- $A$ is the design matrix
- $b$ is the vector of observations
- $\hat{x}$ contains the regression coefficients (intercept and slope)

In [ ]:
function linearRegression_compute(x::Vector{Float64}, y::Vector{Float64}, n::Int)
    # Limit to n points
    x_subset = x[1:n]
    A = hcat(ones(n), x_subset)
    y_subset = y[1:n]
    A_inv = inv(A' * A)
    Q = (A' * y_subset)
    x_hat = A_inv * Q
    return A_inv, x_hat, Q
end

In [ ]:
function linearRegression_plot(x::Vector{Float64}, y::Vector{Float64}, n::Int)
    # Get the computation result
    covar, x_hat, Q = linearRegression_compute(x, y, n)
    #covar, x_hat = randLinearRegression_compute(x, y, n)
    
    # Get the first n points for plotting
    x_subset = x[1:n]
    y_subset = y[1:n]
    
    p = scatter(x_subset, y_subset, label="Data", title="Linear Regression (first $n points)", 
                markersize=5, color=:blue, markerstrokewidth=0)
    plot!(p, x_subset, x_hat[1] .+ x_hat[2] .* x_subset, label="Regression", 
          color=:green, linewidth=2)
    
    return covar, x_hat,Q, p 
end

### Testing Linear Regression

Let's generate some test data and run the linear regression:

In [ ]:
# Generate test data
points = rand(Float64, 16,2) * 10
x = points[:,1]
y = points[:,2]

# Run linear regression on first 10 points
n = 10
#A_inv, x_hat, Q = linearRegression_compute(x, y, n)
covar, x_hat,Q, p = linearRegression_plot(x, y, n)

println("Regression coefficients:")
println("Intercept: ", x_hat[1])
println("Slope: ", x_hat[2])

# Plot linear regression result
display(p)

## 2. Recursive Least Squares

### Mathematical Formulation

The recursive least squares method updates the solution when new data points arrive. The key formulas are:

$$(A^TA+r^Tr)x=A^Tb+r^Tb_{n+1}$$

$$c = \frac{1}{1+r(A^TA)^{-1}r^T}$$

$$\text{covar} = (A^TA)^{-1}$$

$$(A^TA+r^Tr)^{-1}=(A^TA)^{-1}-c*(A^TA)^{-1}r^t*r(A^TA)^{-1}$$

Where:
- $r$ is the new data point
- $c$ is the correction factor
- $\text{covar}$ is the covariance matrix

In [ ]:
function recursiveRegression_compute(x::Vector{Float64}, y::Vector{Float64}, n::Int)
    # Initial regression with first n points
    covar, x_hat, Q = linearRegression_compute(x, y, n)

    for iter in n+1:length(x)
        x_new = [1; x[iter]]  # Column vector
        y_new = y[iter]
        
        gain_vector = covar * x_new
        c = 1 / (1 + (x_new' * gain_vector))
        covar = covar - c * gain_vector * gain_vector'
        Q = Q + (x_new * y_new)
        x_hat = covar * Q
    end
    return covar, x_hat
end

In [ ]:
function recursiveRegression_plot(x::Vector{Float64}, y::Vector{Float64}, n::Int)
    # Initial regression with first n points
    covar, x_hat, Q = linearRegression_compute(x, y, n)
    
    # Create first plot for initial fit
    p1 = scatter(x[1:n], y[1:n], label="Data", title="Linear Regression (first $n points)", 
                markersize=5, color=:blue, markerstrokewidth=0)
    plot!(p1, x[1:n], x_hat[1] .+ x_hat[2] .* x[1:n], label="Regression", 
          color=:green, linewidth=2)

    len = length(x)

    p2 = plot(title="Recursive Least Squares Evolution", legend=:outertopright)
    scatter!(p2, x[1:n], y[1:n], label="Training Data (1-$n)", 
             markersize=5, color=:blue, markerstrokewidth=0)
    
    # New points (n+1 to end) in orange
    scatter!(p2, x[n+1:end], y[n+1:end], label="New Data ($(n+1)-$len)", 
             markersize=5, color=:orange, markershape=:diamond, markerstrokewidth=0)

    # Plot initial model
    initial_x = range(minimum(x), maximum(x), length=100)
    initial_line = x_hat[1] .+ x_hat[2] .* initial_x
    plot!(p2, initial_x, initial_line, label="Initial (n=$n)", linewidth=2, linestyle=:dash, markerstrokewidth=0)

    for iter in n+1:length(x)
        x_new = [1; x[iter]]  # Column vector
        y_new = y[iter]
        
        gain_vector = covar * x_new
        c = 1 / (1 + (x_new' * gain_vector))
        covar = covar - c * gain_vector * x_new' * covar
        error = y_new - (x_new' * x_hat)
        
        x_hat = x_hat + c * error * gain_vector
        
        # Plot this iteration's line with transparency based on iteration
        alpha = 0.4 + 0.6 * (iter - n) / (len - n)  # Increasing opacity
        iteration_line = x_hat[1] .+ x_hat[2] .* initial_x
        if iter % 2 == 0  # Plot every two iterations
            plot!(p2, initial_x, iteration_line, 
                  label="After point $iter", 
                  linewidth=1.5,
                  alpha=alpha)
        end
    end

    #y = b + mx
    final_line = x_hat[1] .+ ( x_hat[2] .* initial_x)
    plot!(p2, initial_x, final_line, label="Final model", linewidth=3, color=:red)
    
    return covar, x_hat, p2
end

### Testing Recursive Least Squares

Let's test the recursive implementation:

In [ ]:
# Run recursive regression
#covar_recursive, x_hat_recursive = recursiveRegression_compute(x, y, n)
covar_recursive, x_hat_recursive, p2 = recursiveRegression_plot(x, y, n)

println("Recursive Regression coefficients:")
println("Intercept: ", x_hat_recursive[1])
println("Slope: ", x_hat_recursive[2])

# Plot recursive regression result
display(p2)

## 3. Kalman Filter Regression

### Mathematical Formulation

The Kalman (batch) filter approach uses:

$$\text{measurement\_covar} := V = I(n) \rightarrow n = \text{points per batch}$$

$$\text{error\_measurement\_covar} := W_0 = (A_0^TV_0^{-1}A_0)^{-1} == covar^{-1}$$

$$W_1^{-1} = W_0^{-1} + A_1^TV_1^{-1}A_1$$

$$\text{Kalman gain} := K_1 = W_1A_1^TV_1^{-1}$$

$$\hat{x}_1 = \hat{x}_0 + K_1(b_1 - A_1 \hat{x}_0)$$

In [ ]:
function CovarRecurrentRegression_compute(x::Vector{Float64}, y::Vector{Float64}, n::Int)
    covar, x_hat, Q = linearRegression_compute(x, y, n)
    
    x_subset = x[n+1:end]
    len = length(x_subset)
    current_A = hcat(ones(len), x_subset)
    y_subset = y[n+1:end]

    inv_measurement_covar = I(len)  # Identity matrix as measurement covariance
    error_covar = covar^-1
    error_covar = error_covar + (current_A' * inv_measurement_covar * current_A)
    kalman_gain = (error_covar^-1) * current_A' * inv_measurement_covar
    x_hat = x_hat + kalman_gain * (y_subset - current_A * x_hat) 
    return error_covar, x_hat
end

In [ ]:
function CovarRecurrentRegression_plot(x::Vector{Float64}, y::Vector{Float64}, n::Int)
    # Call computation function
    covar, x_hat = CovarRecurrentRegression_compute(x, y, n)
    #covar, x_hat = randCovarRecurrentRegression_compute(x, y, n)
    #covar, x_hat = KalmanRecurrentRegression_compute(x, y, n)
    
    #println("Kalman Filter Coefficients: Intercept: ", x_hat[1], ", Slope: ", x_hat[2])
    
    # Plotting
    p = scatter(x, y, label="Data", title="Kalman Filter Regression", 
                markersize=5, color=:blue, markerstrokewidth=0)
    plot!(p, x, x_hat[1] .+ x_hat[2] .* x, label="Kalman Filter Regression", 
          color=:green, linewidth=2)
          
    return covar, x_hat, p
end

### Testing Kalman Filter Regression

Let's test the Kalman filter implementation:

In [ ]:
# Run Kalman filter regression
#error_covar, x_hat_kalman = CovarRecurrentRegression_compute(x, y, n)
kalman_covar, x_hat_kalman, p_kalman = CovarRecurrentRegression_plot(x, y, n)

println("Kalman Filter Regression coefficients:")
println("Intercept: ", x_hat_kalman[1])
println("Slope: ", x_hat_kalman[2])



display(p_kalman)

## 4. Visualization and Comparison

Let's create visualizations to compare all three methods:

In [ ]:
function plotAllRegressionsDetailed(x::Vector{Float64}, y::Vector{Float64}, n::Int)
      # 1. Linear
      _, _, _, p1 = linearRegression_plot(x, y, n)
      title!(p1, "Linear Regression (first $n points)")
  
      # 2. Recursive
      _, _, p2 = recursiveRegression_plot(x, y, n)
      title!(p2, "Recursive Least Squares Evolution")
  
      # 3. Kalman
      _, _, p3 = CovarRecurrentRegression_plot(x, y, n)
      title!(p3, "Kalman Filter Regression")
  
      # 4. Direct comparison of all three fits
      p4 = plot(title="Comparison of All Methods", legend=:outertopright)
      scatter!(p4, x, y, label="All Data Points", markersize=5, color=:black, alpha=0.6)
  
      # grab each method's coefficients again
      covar_lin,     x_hat     , _ = linearRegression_compute(x, y, n)
      covar_rec,     rec_hat   = recursiveRegression_compute(x, y, n)
      covar_kalman,  kalman_hat = CovarRecurrentRegression_compute(x, y, n)
  
      # Print regression coefficients
      println("\nRegression Coefficients:")
      println("Method     | Intercept    | Slope")
      println("-----------|--------------|-------------")
      println("Linear     | $(round(x_hat[1], digits=4)) | $(round(x_hat[2], digits=4))")
      println("Recursive  | $(round(rec_hat[1], digits=4)) | $(round(rec_hat[2], digits=4))")
      println("Kalman     | $(round(kalman_hat[1], digits=4)) | $(round(kalman_hat[2], digits=4))")
  
      plot_x = range(minimum(x), maximum(x), length=100)
      plot!(p4, plot_x, x_hat[1] .+ x_hat[2] .* plot_x,       label="Linear (n=$n)", linewidth=2)
      plot!(p4, plot_x, rec_hat[1] .+ rec_hat[2] .* plot_x,   label="Recursive",       linewidth=2)
      plot!(p4, plot_x, kalman_hat[1] .+ kalman_hat[2] .* plot_x, label="Kalman",       linewidth=2, linestyle=:dash)
  
      final_plot = plot(p1, p2, p3, p4, layout=(2,2), size=(1500,1500), margin=5Plots.mm)
      return final_plot
  end

In [ ]:
# Display the comparison plot
comparison_plot = plotAllRegressionsDetailed(x, y, n)
display(comparison_plot)

## 5. Performance Benchmarking

Let's compare the performance of all three methods:

In [ ]:
function benchmarkRegressionMethods(x::Vector{Float64}, y::Vector{Float64}, n::Int; runs::Int=100, warmup::Int=5)
    # Prepare storage for timing results
    times_linear = Float64[]
    times_recursive = Float64[]
    times_kalman = Float64[]
    times_builtin = Float64[]
    
    # Prepare the design matrix for built-in regression once
    X = hcat(ones(length(x)), x)
    
    # Run warm-up iterations to trigger JIT compilation
    for i in 1:warmup
        linearRegression_compute(x, y, n)
        recursiveRegression_compute(x, y, n)
        CovarRecurrentRegression_compute(x, y, n)
        X \ y
    end
    
    # Main benchmark loop
    for i in 1:runs
        # Force GC before each measurement to reduce variability
        GC.gc()
        
        # 1. Linear Regression
        push!(times_linear, @elapsed linearRegression_compute(x, y, n))

        #GC.gc()
        
        # 2. Recursive Least Squares
        push!(times_recursive, @elapsed recursiveRegression_compute(x, y, n))
        
        #GC.gc()

        # 3. Kalman Filter
        push!(times_kalman, @elapsed CovarRecurrentRegression_compute(x, y, n))
        #push!(times_kalman, @elapsed KalmanRecurrentRegression_compute(x, y, n))
        
        # 4. Built-in regression
        push!(times_builtin, @elapsed X \ y)
    end
    
    # Calculate statistics
    stats_df = DataFrame(
        Method = ["Linear", "Recursive", "Kalman", "Built-in"],
        Min_ms = [minimum(times_linear), minimum(times_recursive),
                 minimum(times_kalman), minimum(times_builtin)] .* 1000,
        Mean_ms = [mean(times_linear), mean(times_recursive),
                  mean(times_kalman), mean(times_builtin)] .* 1000,
        Median_ms = [median(times_linear), median(times_recursive),
                    median(times_kalman), median(times_builtin)] .* 1000,
        Max_ms = [maximum(times_linear), maximum(times_recursive),
                 maximum(times_kalman), maximum(times_builtin)] .* 1000,
        StdDev_ms = [std(times_linear), std(times_recursive),
                    std(times_kalman), std(times_builtin)] .* 1000
    )
    
    # Find the fastest method
    fastest_idx = argmin(stats_df.Median_ms)
    fastest_method = stats_df.Method[fastest_idx]
    
    # Print the usual benchmark table
    println("\n=== Performance Benchmark (over $runs runs with $warmup warmup iterations) ===")
    println("Method | Min (ms) | Mean (ms) | Median (ms) | Max (ms) | StdDev (ms)")
    println("---------|-----------|-----------|-------------|-----------|------------")
    for r in eachrow(stats_df)
        println("$(lpad(r.Method,8)) | $(rpad(round(r.Min_ms, digits=3),9)) | " *
                "$(rpad(round(r.Mean_ms, digits=3),9)) | " *
                "$(rpad(round(r.Median_ms, digits=3),11)) | " *
                "$(rpad(round(r.Max_ms, digits=3),9)) | " *
                "$(round(r.StdDev_ms, digits=3))")
    end
    
    # Create enhanced box + violin plot
    p = boxplot(["Linear" "Recursive" "Kalman" "Built-in"],
              [times_linear .* 1000 times_recursive .* 1000 times_kalman .* 1000 times_builtin .* 1000],
              title="Performance Comparison (lower is better)",
              xlabel="Method", ylabel="Time (ms)",
              linewidth=1.5, fillalpha=0.75, outliers=true, legend=false,
              xtickfontsize=10, ytickfontsize=10, titlefontsize=12)
    
    violin!(p, ["Linear" "Recursive" "Kalman" "Built-in"],
          [times_linear .* 1000 times_recursive .* 1000 times_kalman .* 1000 times_builtin .* 1000],
          alpha=0.3)
    
    # Force y-axis to start at zero for better comparison
    ylims!(p, 0, maximum([maximum(times_linear), maximum(times_recursive), 
                         maximum(times_kalman), maximum(times_builtin)]) * 1000 * 1.1)
    
    # Performance analysis ratios
    mean_times = stats_df.Mean_ms
    linear_time = mean_times[1]
    recursive_time = mean_times[2]
    kalman_time = mean_times[3]
    builtin_time = mean_times[4]
    
    # Corrected performance analysis
    println("\n=== Performance Analysis ===")
    recursive_kalman_ratio = kalman_time / recursive_time
    if recursive_kalman_ratio < 1
        println("- Kalman is $(round(1/recursive_kalman_ratio, digits=1))x faster than Recursive")
    else
        println("- Recursive is $(round(recursive_kalman_ratio, digits=1))x faster than Kalman")
    end
    
    fastest_custom = minimum([recursive_time, kalman_time])
    builtin_custom_ratio = builtin_time / fastest_custom
    if builtin_custom_ratio < 1
        println("- Built-in is $(round(1/builtin_custom_ratio, digits=1))x faster than the fastest custom method")
    else
        println("- Fastest custom method is $(round(builtin_custom_ratio, digits=1))x faster than Built-in")
    end
    
    println("- Fastest method overall: $(fastest_method)")
    
    # Normalize times relative to the fastest method
    fastest_time = minimum(mean_times)
    stats_df.Relative_Speed = stats_df.Mean_ms ./ fastest_time
    
    println("\n=== Relative Performance ===")
    for r in eachrow(stats_df)
        println("$(lpad(r.Method,8)): $(round(r.Relative_Speed, digits=2))x (relative to fastest)")
    end
    
    return p, stats_df
end

In [ ]:
# Run benchmark
benchmark_plot, stats = benchmarkRegressionMethods(x, y, n, runs=50)
display(benchmark_plot)